<a href="https://colab.research.google.com/github/bement2050/Machine_learning_challenge/blob/main/Pok%C3%A9mon_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#wget is used to download files from the servr 
pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=e45f82a11164a84148aa99508ca1769219e4824121b24522d4b00deea9c67531
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [5]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import requests
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow_hub as hub
import wget
from tensorflow.keras.models import load_model


In [ ]:
# There are 899 different types of pokemon the pokeapi website. Get requets are sent  retrivest the types by parsing the json file. 


In [6]:
# send get request and store types of the 899 pokemons
ypes=[requests.get("https://pokeapi.co/api/v2/pokemon/{}".format(i)).json()["types"][0]['type']['name'] for i in list(range(1,899))]

In [7]:
# grouping pokemons by type
pd.Series(types).value_counts()

water       123
normal      109
grass        86
bug          75
fire         58
psychic      58
rock         50
electric     49
dark         36
fighting     36
poison       35
ground       35
dragon       31
ghost        31
steel        30
ice          28
fairy        21
flying        7
dtype: int64

In [8]:
# For this problem I am going to use their front default image to classify their type
# send a get request to the api and retrive Urls for the front default picture  of each pokemon
images=[requests.get("https://pokeapi.co/api/v2/pokemon/{}".format(i)).json()['sprites']['front_default'] for i in list(range(1,899))]

In [9]:
# create a dataframe with two columns  
dataset1=pd.DataFrame({'images':images,'type':types})

In [11]:
#makes a directory to store the images
os.mkdir('dataset')

In [12]:
# creates subdirectories for each unique type under dataset
for x in dataset1['type'].unique():
  os.mkdir('dataset/{}'.format(x))

In [13]:
# Goes through each pokmen on saves the image in to respective directory 
for k in range(len(dataset1)):
  wget.download(dataset1.loc[k]['images'],"dataset/{}".format(dataset1.loc[k]['type']))

In [14]:
# Define some parameters for the loader:


batch_size = 10
img_height =255
img_width = 255

In [15]:
# load these images off disk using the helpful tf.keras.utils.image_dataset_from_directory utility. This will take us  
# from a directory of images on disk to a tf.data.Dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/dataset',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 898 files belonging to 18 classes.
Using 719 files for training.


In [16]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/dataset',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 898 files belonging to 18 classes.
Using 179 files for validation.


In [17]:
# You can find the class names in the class_names attribute on these datasets. These correspond to the directory names 
# in alphabetical order.
type_names=train_ds.class_names

In [18]:
type_names

['bug',
 'dark',
 'dragon',
 'electric',
 'fairy',
 'fighting',
 'fire',
 'flying',
 'ghost',
 'grass',
 'ground',
 'ice',
 'normal',
 'poison',
 'psychic',
 'rock',
 'steel',
 'water']

In [19]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(10, 255, 255, 3)
(10,)


In [20]:
#using buffered prefetching so we can yield data from disk without having I/O become blocking.
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [21]:
# standardizing pixel values between 0 and 1
normalization_layer = layers.Rescaling(1./255)

In [22]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# making pixel values between[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [23]:
num_classes = len(type_names)

In [24]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
  ]
)

In [25]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [26]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 255, 255, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 255, 255, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 255, 255, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 127, 127, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                  

In [28]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15
72/72 [==============================] - 55s 742ms/step - loss: 2.8326 - accuracy: 0.1280 - val_loss: 2.6711 - val_accuracy: 0.1676
Epoch 2/15
72/72 [==============================] - 52s 724ms/step - loss: 2.6140 - accuracy: 0.2197 - val_loss: 2.5910 - val_accuracy: 0.2458
Epoch 3/15
72/72 [==============================] - 52s 726ms/step - loss: 2.5122 - accuracy: 0.2267 - val_loss: 2.5780 - val_accuracy: 0.2570
Epoch 4/15
72/72 [==============================] - 52s 721ms/step - loss: 2.4349 - accuracy: 0.2740 - val_loss: 2.5395 - val_accuracy: 0.2402
Epoch 5/15
72/72 [==============================] - 52s 721ms/step - loss: 2.3639 - accuracy: 0.2851 - val_loss: 2.4966 - val_accuracy: 0.2793
Epoch 6/15
72/72 [==============================] - 52s 726ms/step - loss: 2.3314 - accuracy: 0.3046 - val_loss: 2.5361 - val_accuracy: 0.2179
Epoch 7/15
72/72 [==============================] - 52s 725ms/step - loss: 2.2349 - accuracy: 0.3171 - val_loss: 2.5603 - val_accuracy: 0.2961

In [ ]:
# we got accuracy of 40% on the training set and 25 % on the validation , It is not bad considering we only have few dataset and
# so many classes 

In [29]:
# adding final layer to change the prediction to probability
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [87]:
probability_model.save('mymodel.h5')

In [73]:
probability_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 18)                7898546   
                                                                 
 softmax (Softmax)           (None, 18)                0         
                                                                 
Total params: 7,898,546
Trainable params: 7,898,546
Non-trainable params: 0
_________________________________________________________________


In [74]:
model2=load_model('mymodel.h5')

OSError: ignored

In [111]:
model2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (None, 18)                7898546   
                                                                 
 softmax_1 (Softmax)         (None, 18)                0         
                                                                 
Total params: 7,898,546
Trainable params: 7,898,546
Non-trainable params: 0
_________________________________________________________________


In [91]:
# function to process input . It takes json input ( dict) and outputs  array 
def process(inputs):
  url=inputs['sprites']['front_default']
  url_path = tf.keras.utils.get_file( origin=url)
  img = tf.keras.utils.load_img(url_path, target_size=(img_height, img_width))
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)
  return img_array
# following function takes image as an array and predicts  the type .
def predict(Processed_input):
  predictions = model.predict(Processed_input)
  score = tf.nn.softmax(predictions[0])
  return (type_names[np.argmax(score)])


Example

In [97]:
# Get the data from PokeAPI
resp = requests.get("https://pokeapi.co/api/v2/pokemon/kingdra").json()

In [98]:
processed_resp=process(resp)

16384/816 [==========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [99]:
predict(processed_resp)

'water'